In [64]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import mean_squared_error
import optuna

In [65]:
random.seed(42)
np.random.seed(42)
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [66]:
summaries = pd.read_csv('../data_raw/summaries_train.csv')

In [67]:
summaries = summaries.drop('text', axis=1)

In [68]:
oof_attn_pool = pd.read_csv('../oof_predictions/lgb_attn_pool.csv')
oof_conc_pool = pd.read_csv('../oof_predictions/lgb_conc_pool.csv')
oof_gem_pool = pd.read_csv('../oof_predictions/lgb_gem_pool.csv')
oof_lstm_pool = pd.read_csv('../oof_predictions/lgb_lstm_pool.csv')
oof_mean_pool = pd.read_csv('../oof_predictions/lgb_mean_pool.csv')
oof_meanmax_pool = pd.read_csv('../oof_predictions/lgb_meanmax_pool.csv')
oof_gru_pool = pd.read_csv('../oof_predictions/lgb_gru_pool.csv')

In [69]:
oof_attn_pool.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'summary_length', 'splling_err_num', 'prompt_question', 'prompt_title',
       'prompt_text', 'prompt_length', 'length_ratio', 'word_overlap_count',
       'word_overlap_ratio', 'bigram_overlap_count', 'bigram_overlap_ratio',
       'trigram_overlap_count', 'trigram_overlap_ratio', 'quotes_count',
       'number_of_unique_words', 'flesch_kinsaid', 'flesch', 'gunning_fog',
       'coleman_liau', 'dale_chall', 'ari', 'linsear_write', 'smog', 'spache',
       'pred_content', 'pred_wording', 'pred_lgb_content', 'pred_lgb_wording'],
      dtype='object')

In [70]:
oof_attn_pool = oof_attn_pool.rename({
    'pred_lgb_content': 'content_lgb_attn_pool', 
    'pred_lgb_wording': 'wording_lgb_attn_pool'
}, axis=1)[['student_id', 'prompt_id', 'content_lgb_attn_pool', 'wording_lgb_attn_pool']].copy()
oof_conc_pool = oof_conc_pool.rename({
    'pred_lgb_content': 'content_lgb_conc_pool',
    'pred_lgb_wording': 'wording_lgb_conc_pool'
}, axis=1)[['student_id', 'prompt_id', 'content_lgb_conc_pool', 'wording_lgb_conc_pool']].copy()

oof_gem_pool = oof_gem_pool.rename({
    'pred_lgb_content': 'content_lgb_gem_pool',
    'pred_wording': 'wording_lgb_gem_pool'
}, axis=1)[['student_id', 'prompt_id', 'content_lgb_gem_pool', 'wording_lgb_gem_pool']].copy()

oof_lstm_pool = oof_lstm_pool.rename({
    'pred_lgb_content': 'content_lgb_lstm_pool',
    'pred_lgb_wording': 'wording_lgb_lstm_pool'
}, axis=1)[['student_id', 'prompt_id', 'content_lgb_lstm_pool', 'wording_lgb_lstm_pool']].copy()

oof_mean_pool = oof_mean_pool.rename({
    'pred_lgb_content': 'content_lgb_mean_pool',
    'pred_lgb_wording': 'wording_lgb_mean_pool'
}, axis=1)[['student_id', 'prompt_id', 'content_lgb_mean_pool', 'wording_lgb_mean_pool']].copy()

oof_meanmax_pool = oof_meanmax_pool.rename({
    'pred_lgb_content': 'content_lgb_meanmax_pool',
    'pred_lgb_wording': 'wording_lgb_meanmax_pool'
}, axis=1)[['student_id', 'prompt_id', 'content_lgb_meanmax_pool', 'wording_lgb_meanmax_pool']].copy()
oof_gru_pool = oof_gru_pool.rename({
    'pred_lgb_content': 'content_lgb_gru_pool',
    'pred_lgb_wording': 'wording_lgb_gru_pool'
}, axis=1)[['student_id', 'prompt_id', 'content_lgb_gru_pool', 'wording_lgb_gru_pool']].copy()

In [71]:
summaries = summaries.merge(oof_attn_pool, on=['student_id', 'prompt_id']).merge(
    oof_conc_pool, on=['student_id', 'prompt_id']
).merge(oof_gem_pool, on=['student_id', 'prompt_id']).merge(
    oof_lstm_pool, on=['student_id', 'prompt_id']
).merge(oof_mean_pool, on=['student_id', 'prompt_id']).merge(
    oof_meanmax_pool, on=['student_id', 'prompt_id']).merge(
    oof_gru_pool, on=['student_id', 'prompt_id'])

In [72]:
summaries

,student_id,prompt_id,content,wording,content_lgb_attn_pool,wording_lgb_attn_pool,content_lgb_conc_pool,wording_lgb_conc_pool,content_lgb_gem_pool,wording_lgb_gem_pool,content_lgb_lstm_pool,wording_lgb_lstm_pool,content_lgb_mean_pool,wording_lgb_mean_pool,content_lgb_meanmax_pool,wording_lgb_meanmax_pool,content_lgb_gru_pool,wording_lgb_gru_pool
0,000e8c3c7ddb,814d6b,0.205683,0.380538,-0.097916,0.734486,-0.022150,0.889013,-0.028655,0.612966,-0.552838,0.882381,-0.072543,0.508240,-0.104374,0.499156,0.084610,0.847228
1,0020ae56ffbf,ebad26,-0.548304,0.506755,-0.672357,-0.467967,-0.741401,-0.284348,-0.828224,-0.671266,-0.630435,-0.463764,-0.835795,-1.033750,-0.603110,-0.403993,-0.489004,-0.241624
2,004e978e639e,3b9047,3.128928,4.231226,2.712829,3.050318,2.709813,1.870987,2.654117,2.420776,3.000174,2.709715,2.371794,2.464215,2.905802,2.613999,2.706548,2.796522
3,005ab0199905,3b9047,-0.210614,-0.471415,-0.959623,-0.909390,-1.087302,-0.614463,-1.125971,-0.743153,-1.013784,-0.852298,-1.073987,-0.686038,-1.190612,-0.778618,-1.140000,-1.072360
4,0070c9e7af47,814d6b,3.272894,3.219757,2.468769,2.300629,2.499820,2.718277,2.406918,1.780585,2.221119,2.661260,2.281526,2.441904,2.269914,2.358340,2.909671,2.382062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,0.205683,0.380538,-0.395920,-0.481600,-0.134670,-0.120638,-0.156256,-0.119196,-0.106166,-0.350295,-0.162052,-0.546835,-0.044175,-0.058205,-0.159989,-0.442492
7161,ffc34d056498,3b9047,-0.308448,0.048171,-0.236828,-0.199676,-0.149250,0.096430,-0.473825,-0.274833,-0.165051,-0.146018,-0.437824,-0.314785,-0.521858,0.163570,-0.275630,-0.311119
7162,ffd1576d2e1b,3b9047,-1.408180,-0.493603,-0.649789,-0.187626,-0.877513,-0.333544,-0.857657,-0.696844,-0.386624,-0.075493,-0.977169,-0.214414,-0.965457,-0.513426,-0.877630,-0.434478
7163,ffe4a98093b2,39c16e,-0.393310,0.627128,-0.037286,0.360702,-0.043220,0.623580,-0.178211,0.357063,-0.095070,0.387134,-0.181079,0.479457,-0.076141,0.432329,-0.140196,0.326886


In [73]:
## content weights

In [74]:
summaries['content'].shape

(7165,)

In [75]:
def content_objective(trial):
    
    # w_attn_pool = trial.suggest_float('w_attn_pool', 0, 1)
    w_conc_pool = trial.suggest_float('w_conc_pool', 0, 1)
    w_gem_pool = trial.suggest_float('w_gem_pool', 0, 1)
    w_mean_pool = trial.suggest_float('w_mean_pool', 0, 1)
    # w_lstm_pool = trial.suggest_float('w_lstm_pool', 0, 1)
    # w_meanmax_pool = trial.suggest_float('w_meanmax_pool', 0, 1)
    w_gru_pool = trial.suggest_float('w_gru_pool', 0, 1)
    
    total_w = w_conc_pool + w_gem_pool + w_mean_pool + w_gru_pool
    # w_attn_pool = w_attn_pool / total_w
    w_conc_pool = w_conc_pool / total_w
    w_gem_pool = w_gem_pool / total_w
    w_mean_pool = w_mean_pool / total_w
    # w_lstm_pool = w_lstm_pool / total_w
    # w_meanmax_pool = w_meanmax_pool / total_w
    w_gru_pool = w_gru_pool / total_w
    
    prediction = summaries['content_lgb_conc_pool'] * w_conc_pool + \
        summaries['content_lgb_gem_pool'] * w_gem_pool + \
        summaries['content_lgb_mean_pool'] * w_mean_pool + \
        summaries['content_lgb_gru_pool'] * w_gru_pool
    
    score = mean_squared_error(summaries['content'], prediction, squared=False)
    
    return score
    

In [76]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))

In [77]:
study.optimize(content_objective, n_trials=1000)

In [78]:
best_weights_content = study.best_params

In [79]:
best_weights_content

{'w_conc_pool': 0.9603201720438452,
 'w_gem_pool': 0.5919958288998725,
 'w_mean_pool': 0.4641641838592819,
 'w_gru_pool': 0.8644170361944371}

In [80]:
total_content = 0
for k, v in best_weights_content.items():
    total_content = total_content + v

for k, v in best_weights_content.items():
    best_weights_content[k] = v / total_content

In [81]:
best_weights_content

{'w_conc_pool': 0.33334065687749836,
 'w_gem_pool': 0.20549008988765974,
 'w_mean_pool': 0.16111792551161438,
 'w_gru_pool': 0.30005132772322746}

In [98]:
0.33334065687749836 + 0.20549008988765974 + 0.16111792551161438 + 0.30005132772322746

1.0

In [82]:
for k, v in best_weights_content.items():
    if v < 0.1:
        print(k)

In [83]:
best_score_content = study.best_value

In [84]:
best_score_content

0.4060916575998445

In [85]:
def wording_objective(trial):
    
    w_attn_pool = trial.suggest_float('w_attn_pool', 0, 1)
    w_conc_pool = trial.suggest_float('w_conc_pool', 0, 1)
    # w_gem_pool = trial.suggest_float('w_gem_pool', 0, 1)
    # w_mean_pool = trial.suggest_float('w_mean_pool', 0, 1)
    w_lstm_pool = trial.suggest_float('w_lstm_pool', 0, 1)
    w_meanmax_pool = trial.suggest_float('w_meanmax_pool', 0, 1)
    w_gru_pool = trial.suggest_float('w_gru_pool', 0, 1)
    
    total_w = w_attn_pool + w_conc_pool + w_lstm_pool + w_meanmax_pool + w_gru_pool
    w_attn_pool = w_attn_pool / total_w
    w_conc_pool = w_conc_pool / total_w
    # w_gem_pool = w_gem_pool / total_w
    # w_mean_pool = w_mean_pool / total_w
    w_lstm_pool = w_lstm_pool / total_w
    w_meanmax_pool = w_meanmax_pool / total_w
    w_gru_pool = w_gru_pool / total_w
    
    prediction = summaries['wording_lgb_attn_pool'] * w_attn_pool + \
        summaries['wording_lgb_conc_pool'] * w_conc_pool + \
        summaries['wording_lgb_lstm_pool'] * w_lstm_pool + \
        summaries['wording_lgb_meanmax_pool'] * w_meanmax_pool + \
        summaries['wording_lgb_gru_pool'] * w_gru_pool
    
    score = mean_squared_error(summaries['wording'], prediction, squared=False)
    
    return score

In [86]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))

In [87]:
study.optimize(wording_objective, n_trials=1000)

In [88]:
best_score_wording = study.best_value

In [89]:
best_score_wording

0.5387497063577728

In [90]:
best_weights_wording = study.best_params

In [91]:
best_weights_wording

{'w_attn_pool': 0.31129222641505017,
 'w_conc_pool': 0.3662526920395453,
 'w_lstm_pool': 0.9961902515272636,
 'w_meanmax_pool': 0.47704018347154775,
 'w_gru_pool': 0.5635708898898862}

In [92]:
total_wording = 0
for k, v in best_weights_wording.items():
    total_wording = total_wording + v

for k, v in best_weights_wording.items():
    best_weights_wording[k] = v / total_wording

In [93]:
best_weights_wording

{'w_attn_pool': 0.11468405225695444,
 'w_conc_pool': 0.13493219331828038,
 'w_lstm_pool': 0.3670092767163868,
 'w_meanmax_pool': 0.17574772733635174,
 'w_gru_pool': 0.20762675037202666}

In [94]:
for k, v in best_weights_wording.items():
    if v < 0.1:
        print(k)

In [95]:
## MCRMSE

In [96]:
(best_score_content + best_score_wording)/2

0.4724206819788086

In [97]:
## normalized weights

In [99]:
0.11468405225695444 + 0.13493219331828038 + 0.3670092767163868 + 0.17574772733635174 + 0.20762675037202666

1.0